In [ ]:
dbutils.fs.ls('mnt/olist-bronze/bronze_layer/')

In [ ]:
input_path_trans= '/mnt/olist-bronze/bronze_layer/olist_product_category_name_translation/olist_product_category_name_translation.parquet'
input_path_prod = '/mnt/olist-bronze/bronze_layer/olist_products_dataset/olist_products_dataset.parquet'
input_path_items = '/mnt/olist-bronze/bronze_layer/olist_order_items_dataset/olist_order_items_dataset.parquet'
input_path_orders = '/mnt/olist-bronze/bronze_layer/olist_orders_dataset/olist_orders_dataset.parquet'
input_path_payments = '/mnt/olist-bronze/bronze_layer/olist_order_payments_dataset/olist_order_payments_dataset.parquet'
input_path_customers = '/mnt/olist-bronze/bronze_layer/olist_customers_dataset/olist_customers_dataset.parquet'
input_path_sellers = '/mnt/olist-bronze/bronze_layer/olist_sellers_dataset/olist_sellers_dataset.parquet'
input_path_reviews = '/mnt/olist-bronze/bronze_layer/olist_order_reviews_dataset/olist_order_reviews_dataset.parquet'
input_path_geo = '/mnt/olist-bronze/bronze_layer/olist_geolocation_dataset/olist_geolocation_dataset.parquet'
input_path_census = '/mnt/olist-bronze/bronze_layer/olist_municipalities_pop/olist_municipalities_pop.parquet'

In [ ]:
import pandas as pd
import os
import numpy as np


In [ ]:

columns = {0: "product_category_name", 1: "product_category_name_english"}
product_trans_df = pd.DataFrame(spark.read.format('parquet').load(input_path_trans,header=True).collect())
product_trans_df.rename(columns=columns, inplace=True)
print(product_trans_df.head())
print(product_trans_df.info())

In [ ]:
columns = {0: "product_id", 1: "product_category_name", 2:"product_name_lenght",3: "product_description_lenght", 4: "product_photos_qty", 5: "product_weight_g",6:"product_length_cm",7: "product_height_cm",8:"product_width_cm"}
products_df= pd.DataFrame(spark.read.format('parquet').load(input_path_prod,header=True).collect())
products_df.rename(columns=columns, inplace=True)
products_df.head()

In [ ]:
columns = {0: "order_id", 1: "order_item_id", 2:"product_id",3: "seller_id", 4: "shipping_limit_date", 5: "price",6:"freight_value"}
order_items_df= pd.DataFrame(spark.read.format('parquet').load(input_path_items,header=True).collect())
order_items_df.rename(columns=columns, inplace=True)

In [ ]:
%store order_items_df
print(order_items_df.head())
print(order_items_df.info())

In [ ]:
# Convert the data type of the 'product_category_name','product_category_name_english' column to string
product_trans_df[['product_category_name','product_category_name_english']] = product_trans_df[['product_category_name','product_category_name_english']].astype('string')
print(product_trans_df.isna().sum())
print(product_trans_df.nunique)
print(product_trans_df.describe())
print(product_trans_df.dtypes)

%store product_trans_df

In [ ]:
products_df.info()

In [ ]:
%store -r products_df

# Rename the column
products_df1=products_df.rename(columns={"product_name_lenght": "product_name_length","product_description_lenght": "product_description_length"})

# Filter the DataFrame to find rows where product_weight_g is equal to 0 and null
weight_rows = products_df1[(products_df1["product_weight_g"] == 0) | (products_df1["product_weight_g"].isnull())]

# Replace 0 values in the 'product_weight_g' column with NaN
products_df1['product_weight_g'].replace(0, np.nan, inplace=True)

# Assert that there are no 0 values remaining in the 'product_weight_g' column
assert not (products_df1['product_weight_g'] == 0).any(), "Some 0 values are not replaced with NaN."

# Find rows where product_weight_g is NaN
nan_values = products_df1[products_df1["product_weight_g"].isnull()]["product_weight_g"]

# Change all missing values to NaN
products_df1.replace('', np.nan, inplace=True)
# Replace null values in multiple columns with "NULL" using a dictionary
products_df2 = products_df1.fillna({"product_category_name": "NULL"})

# Convert the data type of the 'product_name_length','product_description_length','product_photos_qty' column to integer
products_df2[['product_name_length','product_description_length','product_photos_qty']] = products_df2[['product_name_length','product_description_length','product_photos_qty']].astype('Int64')

# Convert the data type of the 'product_id','product_category_name' column to string
products_df2[['product_id','product_category_name']] = products_df2[['product_id','product_category_name']].astype('string')

%store products_df2
products_df2.info()


In [ ]:
%store -r products_df2
# Merge the product_trans DataFrame into the products DataFrame based on category names
products_df3 = products_df2.merge(product_trans_df, left_on='product_category_name', right_on='product_category_name', how ="left")
# Display the merged DataFrame
products_df3
# Replace null values in the product_category_name_english column with "NULL" 
products_df3['product_category_name_english'].fillna(value="NULL", inplace=True)
print(products_df3.info())
print(products_df3.nunique())
%store products_df3

In [ ]:
%store -r products_df3

#replace the "_" wtih " " to do standardization of text
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].str.replace("_", " ")

# Specific category names have been replaced with more generalized ones for better clarity and consistency in the data

products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('fashio female clothing', 'fashion female clothing')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('home confort','home comfort')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('costruction tools tools', 'construction tools')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('construction tools safety', 'construction safety tools')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('telephony','telephones')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('construction tools construction', 'construction tools')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('fashion underwear beach', 'fashion beach underwear')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('kitchen dining laundry garden furniture', 'kitchen, dining, laundry and garden furniture')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('fixed telephony', 'fixed telephones')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('books technical','technical books')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('furniture mattress and upholstery','furniture, mattress and upholstery')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('sports leisure', 'sports and leisure')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('books general interest', 'general interest books')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('fashion bags accessories','fashion bags and accessories')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('watches gifts', 'watches and gifts')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('furniture living room', 'living room furniture')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('books imported', 'imported books')
products_df3['product_category_name_english'] = products_df3['product_category_name_english'].replace('health beauty', 'health and beauty')

# Extract unique values from the 'product_category_name_english' column
unique_categories = products_df3['product_category_name_english'].unique()

# Filter the DataFrame based on unique categories
products_df4 = products_df3[products_df3['product_category_name_english'].isin(unique_categories)]

# Print information about the DataFrame
products_df4.info()
print(products_df4['product_category_name_english'])

# Store the DataFrame for later use
%store products_df4

In [ ]:
%store -r order_items_df
# Convert the shipping_limit_date column to datetime format
order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'], format='%Y-%m-%d %H:%M:%S')

# no missing values in the order item
print(order_items_df.isna().sum())

print(order_items_df.nunique())
print(order_items_df.dtypes)
order_items_df1 = order_items_df.copy()
%store order_items_df1

In [ ]:
%store -r order_items_df1
order_items_df1.describe()

In [ ]:
%store -r order_items_df1
# Group by order_id, product_id, seller_id,shipping_limit_date, price and freight_value and count occurrences
grouped_df = order_items_df1.groupby(['order_id', 'product_id', 'seller_id', 'shipping_limit_date','price','freight_value']).size().reset_index(name='count')

# Calculate the total count for each group
order_items_df2 = grouped_df.groupby(['order_id', 'product_id', 'seller_id', 'shipping_limit_date','price','freight_value']).agg({'count': 'sum'}).reset_index()
order_items_df2.rename(columns={'count': 'order_quantity', 'price': 'price_per_item', 'freight_value':'freight_value_per_item'}, inplace=True)

# Sort the total_counts DataFrame by the 'number_of_items_ordered' column in descending order
order_items_df2 = order_items_df2.sort_values(by='order_quantity', ascending=False)

# Reset the index of the sorted DataFrame
order_items_df2.reset_index(drop=True, inplace=True)
%store order_items_df2
# Display the sorted DataFrame with reset index
order_items_df2.head()

In [ ]:
%store -r order_items_df2
print(order_items_df2.nunique())
print(order_items_df2.dtypes)

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
order_items_spark_df = spark.createDataFrame(products_df4)
order_items_spark_df = spark.createDataFrame(order_items_df2)

# Write Spark DataFrame to delta
order_items_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/products")
order_items_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/order_items")

## Transform Orders Data

In [ ]:


order_payments_df1 = pd.DataFrame(spark.read.format("parquet").load(input_path_payments,header=True).collect())

pay_col = {0: "order_id", 
              1: "payment_sequential", 
              2: "payment_type", 
              3: "payment_installments", 
              4: "payment_value"}
order_payments_df1.rename(columns=pay_col, inplace=True) 

order_payments_df1 = order_payments_df1.astype({"order_id": "string",
                                   "payment_sequential": "int", 
                                   "payment_type": "string",
                                   "payment_installments": "int", 
                                   "payment_value": "float"})
order_payments_df1.head() 

In [ ]:
# Get information about the dataset
order_payments_df1.info()

In [ ]:
# Drop 9 rows where payment_value is 0. Remainder: 103,877 
order_payments_df2 = order_payments_df1.copy()
order_payments_df2 = order_payments_df2[order_payments_df2['payment_value'] != 0].reset_index(drop=True)

# Filter the dataframe for a specific order_id
order_id = "fa65dad1b0e818e3ccc5cb0e39231352"
pay_subset = order_payments_df2[order_payments_df2['order_id'] == order_id]

# # Renumber payment_sequential starting from 15
pay_subset2 = pay_subset.copy()
pay_subset2['payment_sequential'] = pay_subset2['payment_sequential'].apply(lambda x: x - 2 if x > 14 else x)

# Drop rows containing the problematic order_id. Remainder: 103,850
order_payments_df3 = order_payments_df2.copy()
order_payments_df3 = order_payments_df3[order_payments_df3['order_id'] != order_id].reset_index(drop=True)

# Then join the remaining dataframe with the renumbered pay_subset2
# Remainder: 103,877 
order_payments_df4 = pd.concat([order_payments_df3, pay_subset2]) 
order_payments_df4.reset_index(drop=True)
order_payments_df4.info()

In [ ]:
orders_df1 = pd.DataFrame(spark.read.format("parquet").load(input_path_orders,header=True).collect())

order_col = {0: "order_id", 
             1: "customer_id", 
             2: "order_status", 
             3: "order_purchase_timestamp", 
             4: "order_approved_at", 
             5: "order_delivered_carrier_date", 
             6: "order_delivered_customer_date", 
             7: "order_estimated_delivery_date"}
orders_df1.rename(columns=order_col, inplace=True) 

orders_df1 = orders_df1.astype({"order_id": "string", 
                                "customer_id": "string", 
                                "order_status": "string"})

# Bhuvana's custom function to parse dates with flexible format handling
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str)
    except ValueError:
        return pd.NaT

# Convert timestamp dtypes 
date_cols = ['order_purchase_timestamp', 'order_approved_at', "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"]
orders_df1[date_cols] = orders_df1[date_cols].applymap(parse_date) 

orders_df1.head() 

In [ ]:
orders_df1.info()

In [ ]:
orders_df2 = orders_df1.copy()

# Calculate the number of days between different datetime column
orders_df2['approved_to_delivered_carrier_days'] = (orders_df2['order_delivered_carrier_date'] - orders_df2['order_approved_at']).dt.days
orders_df2['delivered_carrier_to_customer_days'] = (orders_df2['order_delivered_customer_date'] - orders_df2['order_delivered_carrier_date']).dt.days
orders_df2['purchase_to_delivered_customer_days'] = (orders_df2['order_delivered_customer_date'] - orders_df2['order_purchase_timestamp']).dt.days
orders_df2['estimated_delivery_delay_days'] = (orders_df2['order_delivered_customer_date'] - orders_df2['order_estimated_delivery_date']).dt.days

# Calculate mean, median, and mode for each column
def calculate_stats(column):
    mean = column.mean()
    median = column.median()
    mode = column.mode()[0]
    return mean, median, mode

# Initialize a dictionary to store mode values
mode_values = {}

# Calculate mean, median, and mode for each column
columns = ['approved_to_delivered_carrier_days', 'delivered_carrier_to_customer_days', 'purchase_to_delivered_customer_days', 'estimated_delivery_delay_days']
for col in columns:
    mean, median, mode = calculate_stats(orders_df2[col])
    print(f"\n'{col}':")
    print("Mean:", mean)
    print("Median:", median)
    print("Mode:", mode)
    
    # Store mode value in the dictionary
    mode_values[col] = mode

# Display the mode values dictionary
# Keep them for later aggregation
print("\nMode values dictionary:")
print(mode_values)


In [ ]:
orders_df2.info()

In [ ]:
# Filter rows with 'delivered' order status
delivered_orders = orders_df2[orders_df2['order_status'].isin(['delivered'])]

# Look for missing datetime datatype value
columns_to_fill = ['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']

# Fill missing values for 'delivered' orders with modal values: 
for column in columns_to_fill:
    if column == 'order_approved_at':
        delivered_orders.loc[delivered_orders[column].isnull(), column] = delivered_orders.loc[delivered_orders[column].isnull(), 'order_purchase_timestamp'] + pd.Timedelta(mode_values['approved_to_delivered_carrier_days'], unit='D')
    elif column == 'order_delivered_carrier_date':
        delivered_orders.loc[delivered_orders[column].isnull(), column] = delivered_orders.loc[delivered_orders[column].isnull(), 'order_approved_at'] + pd.Timedelta(mode_values['delivered_carrier_to_customer_days'], unit='D')
    elif column == 'order_delivered_customer_date':
        delivered_orders.loc[delivered_orders[column].isnull(), column] = delivered_orders.loc[delivered_orders[column].isnull(), 'order_delivered_carrier_date'] + pd.Timedelta(mode_values['purchase_to_delivered_customer_days'], unit='D')

delivered_orders.info()

In [ ]:
# Next, drop the "Status: Delivered" records from the original dataset: 2963 records remaining 
orders_df3 = orders_df2.copy()
orders_df3 = orders_df2[orders_df2["order_status"] != "delivered"].reset_index(drop=True)
orders_df3.info()

# Then concatenate the delivered_orders subset back into the original dataset (with all missing values filled)
orders_df4 = pd.concat([orders_df3, delivered_orders])
orders_df4.info()

# Finally, drop the four aggregated fields of timestamp differences as they are now outdated (after filling missing values)
columns_to_drop = ['approved_to_delivered_carrier_days', 
                   'delivered_carrier_to_customer_days', 
                   'purchase_to_delivered_customer_days', 
                   'estimated_delivery_delay_days']
orders_df5 = orders_df4.drop(columns=columns_to_drop)
orders_df5.info()

In [ ]:
# Final step: Add five aggregated fields with updated values, as they will be used for further analysis in Transformation #2 
orders_df6 = orders_df5.copy() 

orders_df6['purchased_to_approved_days'] = (orders_df6['order_approved_at'] - orders_df6['order_purchase_timestamp']).dt.days
orders_df6['approved_to_delivered_carrier_days'] = (orders_df6['order_delivered_carrier_date'] - orders_df6['order_approved_at']).dt.days
orders_df6['delivered_carrier_to_customer_days'] = (orders_df6['order_delivered_customer_date'] - orders_df6['order_delivered_carrier_date']).dt.days
orders_df6['purchased_to_delivered_customer_days'] = (orders_df6['order_delivered_customer_date'] - orders_df6['order_purchase_timestamp']).dt.days
orders_df6['estimated_delivery_delay_days'] = (orders_df6['order_delivered_customer_date'] - orders_df6['order_estimated_delivery_date']).dt.days

orders_df6.info()

In [ ]:
orders_df6.tail()

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
order_payments_spark_df = spark.createDataFrame(order_payments_df4)

# Write Spark DataFrame to delta
order_payments_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/order_payments")

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
orders_spark_df = spark.createDataFrame(orders_df6)

# Write Spark DataFrame to delta
orders_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/orders")

## Transform Geolocation and Sellers Data


In [ ]:
census_df1 = pd.DataFrame(spark.read.format('parquet').load(input_path_census,header=True).collect())

order_col = {0: "municipality", 
             1: "population"
            }
census_df1.rename(columns=order_col, inplace=True) 

census_df1.tail()

In [ ]:
census_df1.info()

In [ ]:
# Split all municipality values into two columns: city and state
# Create a new dataframe with lowercased & whitespace-stripped city, state, and population 

census_df2 = census_df1["municipality"].str.extract(r'(.+)\((\w+)\)').join(census_df1["population"])
census_df2.rename(columns={0: "city", 1: "state"}, inplace=True)
census_df2["city"] = census_df2["city"].str.lower().str.strip()

# Replaced accented characters 
accent_dict = {"á": "a", 
               "é": "e", 
               "í": "i", 
               "ó": "o", 
               "ú": "u", 
               "à": "a", 
               "ò": "o", 
               "â": "a", 
               "ê": "e", 
               "ô": "o", 
               "ã": "a", 
               "õ": "o", 
               "ç": "c", 
               "ü": "u"}

census_converted = census_df2.copy() 
census_converted.replace(accent_dict, regex=True, inplace=True)
census_converted.tail()

census_df3 = census_converted.astype({"city": "string",
                                   "state": "string", 
                                   "population": "int"})
census_df3.tail()

In [ ]:
census_df3.info()

In [ ]:
geolocations_df1 = pd.DataFrame(spark.read.format('parquet').load(input_path_geo,header=True).collect())

geolocations_col = {0: "geolocation_zip_code_prefix", 
                    1: "geolocation_lat", 
                    2: "geolocation_lng", 
                    3: "geolocation_city", 
                    4: "geolocation_state"}
geolocations_df1.rename(columns=geolocations_col, inplace=True) 

geolocations_df1 = geolocations_df1.astype({"geolocation_zip_code_prefix": "int",
                                   "geolocation_lat": "float", 
                                   "geolocation_lng": "float",
                                   "geolocation_city": "string", 
                                   "geolocation_state": "string"})
geolocations_df1.head() 

In [ ]:
geolocations_df1.info()

In [ ]:
# Replaced accented characters 
geolocations_converted = geolocations_df1.copy() 
geolocations_converted.replace(accent_dict, regex=True, inplace=True)
geolocations_converted.tail()
geolocations_converted.info()

In [ ]:
# Drop latitude & longitude outliers (relative to Brazil's range of coordinates)
geolocations_df3 = geolocations_converted.copy() 

geolocations_df3 = geolocations_df3[geolocations_df3["geolocation_lat"] <= 5.288685]
assert geolocations_df3["geolocation_lat"].max() <= 5.288685

geolocations_df3 = geolocations_df3[geolocations_df3["geolocation_lat"] >= -33.798533]
assert geolocations_df3["geolocation_lat"].min() >= -33.798533

geolocations_df3 = geolocations_df3[geolocations_df3["geolocation_lng"] <= -34.703311]
assert geolocations_df3["geolocation_lng"].max() <= -34.703311

geolocations_df3 = geolocations_df3[geolocations_df3["geolocation_lng"] >= -73.968899]
assert geolocations_df3["geolocation_lng"].min() >= -73.968899 

# Group records by zip_code_prefix, getting mean coordinates & modal city and state names 
# Then reset index to make zip_code_prefix a column 
geolocations_df4 = geolocations_df3.copy()
summaries = {"geolocation_lat": "mean", 
             "geolocation_lng": "mean", 
             "geolocation_city": pd.Series.mode, 
             "geolocation_state": pd.Series.mode}
geolocations_df4 = geolocations_df4.groupby(by="geolocation_zip_code_prefix").agg(summaries)
geolocations_df4.head()
assert len(geolocations_df4) == len(geolocations_df3["geolocation_zip_code_prefix"].unique())
geolocations_df5 = geolocations_df4.reset_index() 
geolocations_df5['geolocation_city'] = geolocations_df5['geolocation_city'].astype('string')

geolocations_df5.head() 

In [ ]:
customers_df1 = pd.DataFrame(spark.read.format('parquet').load(input_path_customers,header=True).collect())

customers_col = {0: "customer_id", 
                    1: "customer_unique_id", 
                    2: "customer_zip_code_prefix", 
                    3: "customer_city", 
                    4: "customer_state"}
customers_df1.rename(columns=customers_col, inplace=True) 

customers_df1 = customers_df1.astype({"customer_id": "string",
                                   "customer_unique_id": "string", 
                                   "customer_zip_code_prefix": "int",
                                   "customer_city": "string", 
                                   "customer_state": "string"})
customers_df1.head()

In [ ]:
customers_df1.info()

In [ ]:
# SOMEWHAT USEFUL: Standardisation #1 Use Geolocations to change names
customers_df2 = customers_df1.copy() 
customers_df2["city"] = customers_df2["customer_city"] 

# Subset of Customers with unstandardised city names: 253 unique city names 
unstan_cities = set(customers_df2["city"]).difference(set(census_df3["city"]))
unstan_df = customers_df2[customers_df2["city"].isin(unstan_cities)]

# Use the collapsed copy of Geolocations with one record per municipality zip code: geolocations_df5
# 5039 Geolocations cities that are official municpalities   
official_cities = set(geolocations_df5["geolocation_city"]).intersection(set(census_df3["city"]))

# Subset of Geolocations that contains official municipality names 
official_geo = geolocations_df5[geolocations_df5["geolocation_city"].isin(official_cities)]
official_geo2 = official_geo[["geolocation_zip_code_prefix", "geolocation_city"]] 
%store official_geo2

# Match the subset to the 293 unstandardised city names of the Customers subset 
customers_subset = unstan_df.merge(official_geo2, 
                            left_on="customer_zip_code_prefix",
                            right_on="geolocation_zip_code_prefix", 
                            how="left",
                            suffixes=("_", ""))
customers_subset.head()

# Created a column "official_city" with all 30 official municipality names having replaced initial names 
customers_subset["geolocation_city"] = customers_subset["geolocation_city"].fillna(customers_subset["city"])
customers_subset2 = customers_subset[["customer_id", 
                        "customer_unique_id", 
                        "customer_zip_code_prefix", 
                        "customer_city", 
                        "customer_state", 
                        "geolocation_city"]]
customers_subset2.rename(columns={"geolocation_city": "official_city"}, inplace=True)
customers_subset2.head()

# Replacing unstandardised city names in Customers dataset with official names, matched by zip code 
customers_subset3 = customers_subset2.copy() 
customers_df3 = customers_df2.copy()
customers_df3["city"].update(customers_df3["customer_id"].map(customers_subset3.set_index("customer_id")["official_city"]))
customers_df3.rename(columns={"city": "official_city"}, inplace=True)

# Summary: Out of the 253 unstandardised Customers' unique city names, 
# 26 have been standardised here - leaving 227 unstandardised. 
len(set(customers_df3["official_city"]).difference(set(census_df3["city"])))

In [ ]:
# Standardisation #2 Replacing 230 unstandardised city names with official demographic city names, 
# then testing with assert statements 

customers_df4 = customers_df3.copy() 
city_dict = {"abrantes": "camacari",
             "adhemar de barros": "terra rica", 
             'agisse': "rancharia",
             'aguas claras': "viamao",
             'alexandra': "paranagua",
             'alexandrita': "iturama",
             'alto alegre do iguacu': "capitao leonidas marques",
             'alto sao joao': "roncador",
             'amanari': "maranguape",
             'amparo da serra': "amparo do serra",
             'andrequice': "tres marias",
             'angelo frechiani': "colatina",
             'angustura': "alem paraiba",
             'anhandui': "campo grande",
             'anta': "sapucaia",
             'antonio pereira': "ouro preto",
             'antunes': "igaratinga",
             'aparecida de monte alto': "monte alto",
             'aparecida de sao manuel': "sao manuel", 
             'araguaia': "marechal floriano",
             'areia branca dos assis': "mandirituba",
             'arembepe': "camacari",
             'aribice': "euclides da cunha",
             'arraial d ajuda': "porto seguro",
             "arraial d'ajuda": "porto seguro",
             'arrozal': "pirai",
             'avelar': "paty do alferes",
             'azurita': "mateus leme",
             'bacaxa': "saquarema",
             'baguari': "governador valadares",
             "bandeirantes d'oeste": "sud mennucci",
             'barao ataliba nogueira': "itapira",
             'barao de juparana': "valenca",
             'barra de sao joao': "casimiro de abreu",
             'barra do tarrachil': "chorrocho",
             'bemposta': "tres rios",
             'biritiba-mirim': "biritiba mirim",
             'bom jesus do querendo': "natividade",
             'bonfim paulista': "ribeirao preto",
             'botelho': "santa adelia",
             'braco do rio': "conceicao da barra",
             'brasopolis': "brazopolis", 
             'cachoeira do brumado': "mariana",
             'cachoeira do campo': "ouro preto",
             'california da barra': "barra do pirai",
             'cambiasca': "sao fidelis",
             'campo alegre de minas': "resplendor",
             'capao da porteira': "viamao",
             'caraiba': "ibimirim",
             'carnaiba do sertao': "juazeiro",
             'catu de abrantes': "lauro de freitas",
             'celina': "alegre",
             'central de santa helena': "divino das laranjeiras",
             'chaveslandia': "santa vitoria",
             'cipo-guacu': "embu-guacu",
             'cocais': "barao de cocais",
             'colonia castrolanda': "castro",
             'conceicao da ibitipoca': "lima duarte",
             'conceicao do formoso': "santos dumont",
             'conrado': "miguel pereira",
             'couto de magalhaes': "couto de magalhaes de minas",
             'cuite velho': "conselheiro pena",
             'desembargador otoni': "diamantina",
             'doce grande': "quitandinha",
             'domiciano ribeiro': "ipameri",
             'engenheiro balduino': "monte aprazivel",
             'engenheiro passos': "resende",
             'espigao': "regente feijo",
             'espigao do oeste': "espigao d'oeste",
             'estevao de araujo': "araponga",
             'florinia': "florinea",
             'fonseca': "alvinopolis", 
             'glaura': "ouro preto",
             'goitacazes': "campos dos goytacazes",
             'governador portela': "miguel pereira",
             'graccho cardoso': "gracho cardoso",
             'grao para': "grao-para",
             'guarapua': "dois corregos",
             'guassusse': "oros",
             'guinda': "diamantina",
             'hidreletrica tucurui': "tucurui",
             'holambra ii': "paranapanema",
             'humildes': "feira de santana",
             'ibiajara': "rio do pires",
             'ibiraja': "itanhem",
             'ibitioca': "campos dos goytacazes",
             'ibitira': "rio do antonio",
             'ibitiuva': "pitangueiras",
             'ilha dos valadares': "paranagua",
             'ipiabas': "barra do pirai",
             'irape': "chavantes",
             'itabatan': "mucuri",
             'itacurussa': "mangaratiba",
             'itaipava': "itapemirim",
             'itapage': "itapaje",
             'jacare': "cabreuva",
             'jacigua': "vargem alta",
             'jaguarembe': "itaocara",
             'jamaica': "dracena",
             'jamapara': "sapucaia",
             'japuiba': "cachoeiras de macacu",
             'jardim abc de goias': "cidade ocidental",
             'jaua': "camacari",
             'lidice': "rio claro",
             'luizlandia do oeste': "sao goncalo do abaete",
             'luziapolis': "campo alegre",
             'macuco de minas': "itumirim",
             'maioba': "paco do lumiar",
             'major porto': "patos de minas",
             'mariental': "lapa",
             'maristela': "laranjal paulista",
             'martinesia': "uberlandia",
             'missi': "iraucuba",
             'mogi-guacu': "mogi guacu",
             'monnerat': "duas barras",
             'monte alverne': "santa cruz do sul",
             'monte bonito': "pelotas",
             'monte gordo': "camacari",
             'monte verde': "camanducaia",
             'morro de sao paulo': "cairu",
             'morro do ferro': "oliveira",
             'morro vermelho': "caete",
             'murucupi': "barcarena",
             'mutum parana': "porto velho",
             'nossa senhora de caravaggio': "nova veneza",
             'nossa senhora do o': "ipojuca",
             'nossa senhora do remedio': "salesopolis",
             "olhos d'agua": "belo horizonte",
             'osvaldo kroeff': "cambara do sul", 
             'pacotuba': "cachoeiro de itapemirim",
             'padre gonzales': "tres passos",
             'palmeirinha': "guarapuava",
             'palmital de minas': "cabeceira grande",
             'papucaia': "cachoeiras de macacu",
             'paraju': "domingos martins",
             'parati': "paraty",
             'passa tres': "rio claro",
             'pedra menina': "rio vermelho",
             'perola independente': "maripa",
             'perpetuo socorro': "belo oriente",
             'piao': "sao jose do vale do rio preto",
             "picarras": "balneario picarras", 
             'pindare mirim': "igarape do meio",
             'pinhotiba': "eugenopolis",
             'pitanga de estrada': "mamanguape",
             'piumhii': "piumhi",
             'poco de pedra': "sao goncalo do amarante",
             'pocoes de paineiras': "paineiras",
             'polo petroquimico de triunfo': "triunfo",
             'ponto do marambaia': "carai",
             'portela': "itaocara",
             'porto trombetas': "oriximina",
             "posto da mata": "nova vicosa", 
             'prudencio thomaz': "rio brilhante",
             'purilandia': "porciuncula",
             'quatro bocas': "tome-acu",
             'queixada': "novo cruzeiro",
             'quilometro 14 do mutum': "baixo guandu",
             'rainha do mar': "xangri-la",
             'raposo': "itaperuna",
             'ravena': "sabara",
             'rechan': "itapetininga",
             'ribeiro junqueira': "leopoldina",
             "sacra familia do tingua": "engenheiro paulo de frontin",
             "salobro": "canarana",
             'santa barbara d oeste': "santa barbara d'oeste", 
             'sanga puita': "ponta pora",
             'santa cruz do prata': "guaranesia",
             'santa cruz do timbo': "porto uniao",
             'santa isabel do para': "santa izabel do para",
             'santa isabel do rio preto': "valenca",
             'santa rita da floresta': "cantagalo",
             'santa rita do ibitipoca': "santa rita de ibitipoca",
             'santana do capivari': "pouso alto",
             'santana do livramento': "sant'ana do livramento",
             'santana do sobrado': "casa nova",
             'santanesia': "pirai",
             'santo antonio das queimadas': "jurema",
             'santo antonio do canaa': "santa teresa",
             'santo antonio do leverger': "santo antonio de leverger", 
             "santo antonio dos campos": "divinopolis",
             "santo eduardo": "campos dos goytacazes", 
             "sao clemente": "santa helena",
             "sao francisco do humaita": "mutum",
             "sao geraldo do baguari": "sao joao evangelista",
             "sao goncalo do rio das pedras": "serro", 
             "sao joao de petropolis": "santa teresa",
             "sao joao do sobrado": "pinheiros",
             "sao jorge do oeste": "sao jorge d'oeste",
             "sao jose do ribeirao": "bom jardim",
             "sao jose do turvo": "barra do pirai",
             "sao mateus de minas": "camanducaia", 
             "sao miguel do cambui": "marialva", 
             "sao sebastiao da serra": "brotas",
             "sao sebastiao de campos": "campos dos goytacazes",
             "sao sebastiao do paraiba": "cantagalo", 
             "sao thome das letras": "tres coracoes",
             "sao vitor": "governador valadares",
             "sede alvorada": "cascavel",
             "serra bonita": "buritis",
             "serra dos dourados": "umuarama",
             "silvano": "patrocinio",
             "silveira carvalho": "barao de monte alto",
             "siriji": "sao vicente ferrer", 
             "sucesso": "tamboril", 
             "taperuaba": "sobral",
             "tapinas": "itapolis", 
             "tecainda": "martinopolis",
             "termas de ibira": "ibira",
             "tocos": "campos dos goytacazes",
             "trancoso": "porto seguro",
             "tres irmaos": "cambuci",
             "tuparece": "medina", 
             "valao do barro": "sao sebastiao do alto",
             "vargem grande do soturno": "cachoeiro de itapemirim",
             "venda branca": "casa branca",
             "vermelho": "muriae",
             "vila dos cabanos": "barcarena",
             "vila muriqui": "mangaratiba",
             "vila nelita": "agua doce do norte",
             "vila nova": "toledo",
             "vila pereira": "nanuque",
             "vila reis": "apucarana",
             "visconde de maua": "resende", 
             "vitorinos": "alto rio doce",
             "werneck": "paraiba do sul", 
             "arace": "domingos martins",
             "brejo bonito": "cruzeiro da fortaleza", 
             'carajas': "canaa dos carajas",
             'conservatoria': "valenca",
             'itamira': "apora",
             'quatituba': "itueta",
             'santo amaro de campos': "campos dos goytacazes",
             'travessao': "campos dos goytacazes"} 
customers_df4.replace({"official_city": city_dict}, inplace=True) 

assert customers_df4[customers_df4["official_city"] == "abrantes"].empty == True
assert customers_df4[customers_df4["official_city"] == "adhemar de barros"].empty == True
assert customers_df4[customers_df4["official_city"] == "agisse"].empty == True 
assert customers_df4[customers_df4["official_city"] == "aguas claras"].empty == True 
assert customers_df4[customers_df4["official_city"] == "alexandra"].empty == True 
assert customers_df4[customers_df4["official_city"] == "alexandrita"].empty == True 
assert customers_df4[customers_df4["official_city"] == "alto alegre do iguacu"].empty == True 
assert customers_df4[customers_df4["official_city"] == "alto sao joao"].empty == True 
assert customers_df4[customers_df4["official_city"] == "amanari"].empty == True 
assert customers_df4[customers_df4["official_city"] == "amparo da serra"].empty == True 
assert customers_df4[customers_df4["official_city"] == "andrequice"].empty == True 
assert customers_df4[customers_df4["official_city"] == "angelo frechiani"].empty == True 
assert customers_df4[customers_df4["official_city"] == "angustura"].empty == True 
assert customers_df4[customers_df4["official_city"] == "anhandui"].empty == True 
assert customers_df4[customers_df4["official_city"] == "anta"].empty == True 
assert customers_df4[customers_df4["official_city"] == "antonio pereira"].empty == True 
assert customers_df4[customers_df4["official_city"] == "antunes"].empty == True 
assert customers_df4[customers_df4["official_city"] == "aparecida de monte alto"].empty == True 
assert customers_df4[customers_df4["official_city"] == "aparecida de sao manuel"].empty == True
assert customers_df4[customers_df4["official_city"] == "araguaia"].empty == True 
assert customers_df4[customers_df4["official_city"] == "areia branca dos assis"].empty == True 
assert customers_df4[customers_df4["official_city"] == "arembepe"].empty == True 
assert customers_df4[customers_df4["official_city"] == "aribice"].empty == True 
assert customers_df4[customers_df4["official_city"] == "arraial d ajuda"].empty == True 
assert customers_df4[customers_df4["official_city"] == "arraial d'ajuda"].empty == True 
assert customers_df4[customers_df4["official_city"] == "arrozal"].empty == True 
assert customers_df4[customers_df4["official_city"] == "avelar"].empty == True 
assert customers_df4[customers_df4["official_city"] == "azurita"].empty == True 
assert customers_df4[customers_df4["official_city"] == "bacaxa"].empty == True 
assert customers_df4[customers_df4["official_city"] == "baguari"].empty == True 
assert customers_df4[customers_df4["official_city"] == "bandeirantes d'oeste"].empty == True 
assert customers_df4[customers_df4["official_city"] == "barao ataliba nogueira"].empty == True 
assert customers_df4[customers_df4["official_city"] == "barao de juparana"].empty == True 
assert customers_df4[customers_df4["official_city"] == "barra de sao joao"].empty == True 
assert customers_df4[customers_df4["official_city"] == "barra do tarrachil"].empty == True 
assert customers_df4[customers_df4["official_city"] == "bemposta"].empty == True 
assert customers_df4[customers_df4["official_city"] == "biritiba-mirim"].empty == True 
assert customers_df4[customers_df4["official_city"] == "bom jesus do querendo"].empty == True 
assert customers_df4[customers_df4["official_city"] == "bonfim paulista"].empty == True 
assert customers_df4[customers_df4["official_city"] == "botelho"].empty == True 
assert customers_df4[customers_df4["official_city"] == "braco do rio"].empty == True 
assert customers_df4[customers_df4["official_city"] == "brasopolis"].empty == True 
assert customers_df4[customers_df4["official_city"] == "cachoeira do brumado"].empty == True 
assert customers_df4[customers_df4["official_city"] == "cachoeira do campo"].empty == True 
assert customers_df4[customers_df4["official_city"] == "california da barra"].empty == True 
assert customers_df4[customers_df4["official_city"] == "cambiasca"].empty == True 
assert customers_df4[customers_df4["official_city"] == "campo alegre de minas"].empty == True 
assert customers_df4[customers_df4["official_city"] == "capao da porteira"].empty == True 
assert customers_df4[customers_df4["official_city"] == "caraiba"].empty == True 
assert customers_df4[customers_df4["official_city"] == "carnaiba do sertao"].empty == True 
assert customers_df4[customers_df4["official_city"] == "catu de abrantes"].empty == True 
assert customers_df4[customers_df4["official_city"] == "celina"].empty == True 
assert customers_df4[customers_df4["official_city"] == "central de santa helena"].empty == True 
assert customers_df4[customers_df4["official_city"] == "chaveslandia"].empty == True 
assert customers_df4[customers_df4["official_city"] == "cipo-guacu"].empty == True 
assert customers_df4[customers_df4["official_city"] == "cocais"].empty == True
assert customers_df4[customers_df4["official_city"] == "colonia castrolanda"].empty == True 
assert customers_df4[customers_df4["official_city"] == "conceicao da ibitipoca"].empty == True 
assert customers_df4[customers_df4["official_city"] == "conceicao do formoso"].empty == True 
assert customers_df4[customers_df4["official_city"] == "conrado"].empty == True 
assert customers_df4[customers_df4["official_city"] == "couto de magalhaes"].empty == True 
assert customers_df4[customers_df4["official_city"] == "cuite velho"].empty == True 
assert customers_df4[customers_df4["official_city"] == "desembargador otoni"].empty == True 
assert customers_df4[customers_df4["official_city"] == "doce grande"].empty == True 
assert customers_df4[customers_df4["official_city"] == "domiciano ribeiro"].empty == True 
assert customers_df4[customers_df4["official_city"] == "engenheiro balduino"].empty == True 
assert customers_df4[customers_df4["official_city"] == "engenheiro passos"].empty == True 
assert customers_df4[customers_df4["official_city"] == "espigao"].empty == True 
assert customers_df4[customers_df4["official_city"] == "espigao do oeste"].empty == True 
assert customers_df4[customers_df4["official_city"] == "estevao de araujo"].empty == True 
assert customers_df4[customers_df4["official_city"] == "florinia"].empty == True 
assert customers_df4[customers_df4["official_city"] == "fonseca"].empty == True 
assert customers_df4[customers_df4["official_city"] == "glaura"].empty == True 
assert customers_df4[customers_df4["official_city"] == "goitacazes"].empty == True 
assert customers_df4[customers_df4["official_city"] == "governador portela"].empty == True 
assert customers_df4[customers_df4["official_city"] == "graccho cardoso"].empty == True 
assert customers_df4[customers_df4["official_city"] == "grao para"].empty == True 
assert customers_df4[customers_df4["official_city"] == "guarapua"].empty == True 
assert customers_df4[customers_df4["official_city"] == "guassusse"].empty == True 
assert customers_df4[customers_df4["official_city"] == "guinda"].empty == True 
assert customers_df4[customers_df4["official_city"] == "hidreletrica tucurui"].empty == True 
assert customers_df4[customers_df4["official_city"] == "holambra ii"].empty == True 
assert customers_df4[customers_df4["official_city"] == "humildes"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ibiajara"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ibiraja"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ibitioca"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ibitira"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ibitiuva"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ilha dos valadares"].empty == True 
assert customers_df4[customers_df4["official_city"] == "ipiabas"].empty == True 
assert customers_df4[customers_df4["official_city"] == "irape"].empty == True 
assert customers_df4[customers_df4["official_city"] == "itabatan"].empty == True 
assert customers_df4[customers_df4["official_city"] == "itacurussa"].empty == True 
assert customers_df4[customers_df4["official_city"] == "itaipava"].empty == True 
assert customers_df4[customers_df4["official_city"] == "itapage"].empty == True 
assert customers_df4[customers_df4["official_city"] == "jacare"].empty == True
assert customers_df4[customers_df4["official_city"] == "jacigua"].empty == True
assert customers_df4[customers_df4["official_city"] == "jaguarembe"].empty == True
assert customers_df4[customers_df4["official_city"] == "jamaica"].empty == True
assert customers_df4[customers_df4["official_city"] == "jamapara"].empty == True
assert customers_df4[customers_df4["official_city"] == "japuiba"].empty == True
assert customers_df4[customers_df4["official_city"] == "jardim abc de goias"].empty == True
assert customers_df4[customers_df4["official_city"] == "jaua"].empty == True
assert customers_df4[customers_df4["official_city"] == "lidice"].empty == True
assert customers_df4[customers_df4["official_city"] == "luizlandia do oeste"].empty == True
assert customers_df4[customers_df4["official_city"] == "luziapolis"].empty == True
assert customers_df4[customers_df4["official_city"] == "macuco de minas"].empty == True
assert customers_df4[customers_df4["official_city"] == "maioba"].empty == True
assert customers_df4[customers_df4["official_city"] == "major porto"].empty == True
assert customers_df4[customers_df4["official_city"] == "mariental"].empty == True
assert customers_df4[customers_df4["official_city"] == "maristela"].empty == True
assert customers_df4[customers_df4["official_city"] == "martinesia"].empty == True
assert customers_df4[customers_df4["official_city"] == "missi"].empty == True
assert customers_df4[customers_df4["official_city"] == "mogi-guacu"].empty == True
assert customers_df4[customers_df4["official_city"] == "monnerat"].empty == True
assert customers_df4[customers_df4["official_city"] == "monte alverne"].empty == True
assert customers_df4[customers_df4["official_city"] == "monte bonito"].empty == True
assert customers_df4[customers_df4["official_city"] == "monte gordo"].empty == True
assert customers_df4[customers_df4["official_city"] == "morro de sao paulo"].empty == True
assert customers_df4[customers_df4["official_city"] == "morro do ferro"].empty == True
assert customers_df4[customers_df4["official_city"] == "morro vermelho"].empty == True
assert customers_df4[customers_df4["official_city"] == "murucupi"].empty == True
assert customers_df4[customers_df4["official_city"] == "mutum parana"].empty == True
assert customers_df4[customers_df4["official_city"] == "nossa senhora de caravaggio"].empty == True
assert customers_df4[customers_df4["official_city"] == "nossa senhora do o"].empty == True
assert customers_df4[customers_df4["official_city"] == "nossa senhora do remedio"].empty == True
assert customers_df4[customers_df4["official_city"] == "olhos d'agua"].empty == True
assert customers_df4[customers_df4["official_city"] == "osvaldo kroeff"].empty == True
assert customers_df4[customers_df4["official_city"] == "pacotuba"].empty == True
assert customers_df4[customers_df4["official_city"] == "padre gonzales"].empty == True
assert customers_df4[customers_df4["official_city"] == "palmeirinha"].empty == True
assert customers_df4[customers_df4["official_city"] == "palmital de minas"].empty == True
assert customers_df4[customers_df4["official_city"] == "papucaia"].empty == True
assert customers_df4[customers_df4["official_city"] == "paraju"].empty == True
assert customers_df4[customers_df4["official_city"] == "parati"].empty == True
assert customers_df4[customers_df4["official_city"] == "passa tres"].empty == True
assert customers_df4[customers_df4["official_city"] == "pedra menina"].empty == True
assert customers_df4[customers_df4["official_city"] == "perola independente"].empty == True
assert customers_df4[customers_df4["official_city"] == "perpetuo socorro"].empty == True
assert customers_df4[customers_df4["official_city"] == "piao"].empty == True
assert customers_df4[customers_df4["official_city"] == "picarras'"].empty == True
assert customers_df4[customers_df4["official_city"] == "pindare mirim"].empty == True
assert customers_df4[customers_df4["official_city"] == "pinhotiba"].empty == True
assert customers_df4[customers_df4["official_city"] == "pitanga de estrada"].empty == True
assert customers_df4[customers_df4["official_city"] == "piumhii"].empty == True
assert customers_df4[customers_df4["official_city"] == "poco de pedra"].empty == True
assert customers_df4[customers_df4["official_city"] == "pocoes de paineiras"].empty == True
assert customers_df4[customers_df4["official_city"] == "polo petroquimico de triunfo"].empty == True
assert customers_df4[customers_df4["official_city"] == "ponto do marambaia"].empty == True
assert customers_df4[customers_df4["official_city"] == "portela"].empty == True
assert customers_df4[customers_df4["official_city"] == "porto trombetas"].empty == True
assert customers_df4[customers_df4["official_city"] == "posto da mata"].empty == True
assert customers_df4[customers_df4["official_city"] == "prudencio thomaz"].empty == True
assert customers_df4[customers_df4["official_city"] == "purilandia"].empty == True
assert customers_df4[customers_df4["official_city"] == "quatro bocas"].empty == True
assert customers_df4[customers_df4["official_city"] == "queixada"].empty == True
assert customers_df4[customers_df4["official_city"] == "quilometro 14 do mutum"].empty == True
assert customers_df4[customers_df4["official_city"] == "rainha do mar"].empty == True
assert customers_df4[customers_df4["official_city"] == "raposo"].empty == True
assert customers_df4[customers_df4["official_city"] == "ravena"].empty == True
assert customers_df4[customers_df4["official_city"] == "rechan"].empty == True
assert customers_df4[customers_df4["official_city"] == "ribeiro junqueira"].empty == True
assert customers_df4[customers_df4["official_city"] == "sacra familia do tingua"].empty == True
assert customers_df4[customers_df4["official_city"] == "salobro"].empty == True
assert customers_df4[customers_df4["official_city"] == "sanga puita"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa barbara d oeste"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa cruz do prata"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa cruz do timbo"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa isabel do para"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa isabel do rio preto"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa rita da floresta"].empty == True
assert customers_df4[customers_df4["official_city"] == "santa rita do ibitipoca"].empty == True
assert customers_df4[customers_df4["official_city"] == "santana do capivari"].empty == True
assert customers_df4[customers_df4["official_city"] == "santana do livramento"].empty == True
assert customers_df4[customers_df4["official_city"] == "santana do sobrado"].empty == True
assert customers_df4[customers_df4["official_city"] == "santanesia"].empty == True
assert customers_df4[customers_df4["official_city"] == "santo antonio das queimadas"].empty == True
assert customers_df4[customers_df4["official_city"] == "santo antonio do canaa"].empty == True
assert customers_df4[customers_df4["official_city"] == "santo antonio do leverger"].empty == True
assert customers_df4[customers_df4["official_city"] == "santo antonio dos campos"].empty == True 
assert customers_df4[customers_df4["official_city"] == "santo eduardo"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao clemente"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao francisco do humaita"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao geraldo do baguari"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao goncalo do rio das pedras"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao joao de petropolis"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao joao do sobrado"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao jorge do oeste"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao jose do ribeirao"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao jose do turvo"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao mateus de minas"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao miguel do cambui"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao sebastiao da serra"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao sebastiao de campos"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao sebastiao do paraiba"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao thome das letras"].empty == True
assert customers_df4[customers_df4["official_city"] == "sao vitor"].empty == True
assert customers_df4[customers_df4["official_city"] == "sede alvorada"].empty == True
assert customers_df4[customers_df4["official_city"] == "serra bonita"].empty == True
assert customers_df4[customers_df4["official_city"] == "serra dos dourados"].empty == True
assert customers_df4[customers_df4["official_city"] == "silvano"].empty == True
assert customers_df4[customers_df4["official_city"] == "silveira carvalho"].empty == True
assert customers_df4[customers_df4["official_city"] == "siriji"].empty == True
assert customers_df4[customers_df4["official_city"] == "sucesso"].empty == True
assert customers_df4[customers_df4["official_city"] == "taperuaba"].empty == True
assert customers_df4[customers_df4["official_city"] == "tapinas"].empty == True
assert customers_df4[customers_df4["official_city"] == "tecainda"].empty == True
assert customers_df4[customers_df4["official_city"] == "termas de ibira"].empty == True
assert customers_df4[customers_df4["official_city"] == "tocos"].empty == True
assert customers_df4[customers_df4["official_city"] == "trancoso"].empty == True
assert customers_df4[customers_df4["official_city"] == "tres irmaos"].empty == True
assert customers_df4[customers_df4["official_city"] == "tuparece"].empty == True
assert customers_df4[customers_df4["official_city"] == "valao do barro"].empty == True
assert customers_df4[customers_df4["official_city"] == "vargem grande do soturno"].empty == True
assert customers_df4[customers_df4["official_city"] == "venda branca"].empty == True
assert customers_df4[customers_df4["official_city"] == "vermelho"].empty == True
assert customers_df4[customers_df4["official_city"] == "vila dos cabanos"].empty == True
assert customers_df4[customers_df4["official_city"] == "vila muriqui"].empty == True
assert customers_df4[customers_df4["official_city"] == "vila nelita"].empty == True
assert customers_df4[customers_df4["official_city"] == "vila nova"].empty == True
assert customers_df4[customers_df4["official_city"] == "vila pereira"].empty == True
assert customers_df4[customers_df4["official_city"] == "vila reis"].empty == True
assert customers_df4[customers_df4["official_city"] == "visconde de maua"].empty == True
assert customers_df4[customers_df4["official_city"] == "vitorinos"].empty == True
assert customers_df4[customers_df4["official_city"] == "werneck"].empty == True
assert customers_df4[customers_df4["official_city"] == "arace"].empty == True
assert customers_df4[customers_df4["official_city"] == "brejo bonito"].empty == True
assert customers_df4[customers_df4["official_city"] == "carajas"].empty == True 
assert customers_df4[customers_df4["official_city"] == "conservatoria"].empty == True 
assert customers_df4[customers_df4["official_city"] == "itamira"].empty == True 
assert customers_df4[customers_df4["official_city"] == "quatituba"].empty == True 
assert customers_df4[customers_df4["official_city"] == "santo amaro de campos"].empty == True 
assert customers_df4[customers_df4["official_city"] == "travessao"].empty == True 

customers_df4.info()

# Summary: All 230 unique city names have been unstandardised 
len(set(customers_df4["official_city"]).difference(set(census_df3["city"])))

In [ ]:
sellers_df1 = pd.DataFrame(spark.read.format('parquet').load(input_path_sellers,header=True).collect())

sellers_col = {0: "seller_id", 
                    1: "seller_zip_code_prefix", 
                    2: "seller_city", 
                    3: "seller_state"}
sellers_df1.rename(columns=sellers_col, inplace=True) 

sellers_df1 = sellers_df1.astype({"seller_id": "string",
                                   "seller_zip_code_prefix": "int",
                                   "seller_city": "string", 
                                   "seller_state": "string"})
sellers_df1.head()

In [ ]:
sellers_df1.info()

In [ ]:
# HIGHLY EFFECTIVE: Standardisation #1 Using Geolocations to change names 
sellers_df2 = sellers_df1.copy()
sellers_df2["city"] = sellers_df2["seller_city"]

# Subset of Customers with unstandardised city names: 71 unique city names 
unstan_cities2 = set(sellers_df2["city"]).difference(set(census_df3["city"]))
unstan_df2 = sellers_df2[sellers_df2["city"].isin(unstan_cities2)]

# Use the subset of Geolocations that contains official municipality names: official_geo2
# Match the subset to the 71 unstardardised city names of the Sellers subset 
sellers_subset = unstan_df2.merge(official_geo2, 
                            left_on="seller_zip_code_prefix",
                            right_on="geolocation_zip_code_prefix", 
                            how="left",
                            suffixes=("_", ""))
sellers_subset.head()

# Created a column "official_city" with all 30 official municipality names having replaced initial names 
sellers_subset["geolocation_city"] = sellers_subset["geolocation_city"].fillna(sellers_subset["city"])
sellers_subset2 = sellers_subset[["seller_id", 
                        "seller_zip_code_prefix", 
                        "seller_city", 
                        "seller_state", 
                        "geolocation_city"]]
sellers_subset2.rename(columns={"geolocation_city": "official_city"}, inplace=True)
sellers_subset2.head()

# Replacing unstandardised city names in Sellers dataset with official names, matched by zip code 
sellers_subset3 = sellers_subset2.copy()
sellers_df3 = sellers_df2.copy()
sellers_df3["city"].update(sellers_df3["seller_id"].map(sellers_subset3.set_index("seller_id")["official_city"]))
sellers_df3.rename(columns={"city": "official_city"}, inplace=True)  

# Summary: Out of the 71 unstandardised Sellers' unique city names, 70 have been standardised here. 
len(set(sellers_df3["official_city"]).difference(set(census_df3["city"])))

In [ ]:
# Standardising #2 Replacing unstandardised city name with official demographic city name, 
# then testing with assert statement 
sellers_df4 = sellers_df3.copy()
sellers_df4.replace({"arraial d'ajuda (porto seguro)": "porto seguro"}, inplace=True) 

assert sellers_df4[sellers_df4["official_city"] == "arraial d'ajuda (porto seguro)"].empty == True

sellers_df4.info()

# Summary: The 1 remaining unstandardised Sellers' unique city name has been standardised. 
len(set(sellers_df4["official_city"]).difference(set(census_df3["city"]))) 

In [ ]:
# Imputation of missing zip_code_prefixes from Sellers & Customers into Geolocations 

import numpy as np

# First, create a list of Sellers' zipcodes to be added (missing from Geolocations) 
zip_sell_list = list(set(sellers_df4["seller_zip_code_prefix"]).difference(set(geolocations_df5["geolocation_zip_code_prefix"])))
zip_sell_list.sort()

# Second, identify each missing zipcode's closest zipcode 

def closest(lst, missing_zip): 
    lst = np.asarray(lst)
    idx = (np.abs(lst - missing_zip)).argmin()
    return lst[idx]

lst = geolocations_df5["geolocation_zip_code_prefix"].tolist()
subs_zip = []
for missing_zip in zip_sell_list: 
    subs_zip.append(closest(lst, missing_zip))
    
# Third, create a dictionary of matching "closest zipcodes" and "missing zipcodes" 
sell_dict = dict(zip(subs_zip, zip_sell_list))

# Fourth, identify the records containing the "closest zipcodes" 
sell_zip_df = geolocations_df5[geolocations_df5["geolocation_zip_code_prefix"].isin(subs_zip)]

# Fifth, create a new dataframe by replacing the "closest zipcodes" with their 
# matching "missing zipcodes" 
sell_zip_df2 = sell_zip_df.copy()
sell_zip_df2["geolocation_zip_code_prefix"] = sell_zip_df2["geolocation_zip_code_prefix"].map(sell_dict)

# Sixth, concatenate the original Geolocations dataframe with the new dataframe 
geolocations_df6 = pd.concat([geolocations_df5, sell_zip_df2])
geolocations_df6.reset_index(drop=True)
# %store geolocations_df6

# Check if there are still Sellers' zipcodes missing from Geolocations
missing_zipcodes0 = list(set(sellers_df4["seller_zip_code_prefix"]).difference(set(geolocations_df6["geolocation_zip_code_prefix"])))
if not missing_zipcodes0:
    print("All sellers' zipcodes are now present in the Geolocations dataframe.")
else:
    print(f"The following {len(missing_zipcodes0)} zipcodes are still missing from the Geolocations dataframe: {missing_zipcodes0}")


In [ ]:
# The no. of unique geolocations_zip_code_prefix has increased by 7 (from Sellers) to 19,017
geolocations_df6.nunique()

In [ ]:
# Create a list of customers' zipcodes that are missing from the Geolocations dataframe
zip_cust_list = list(set(customers_df4["customer_zip_code_prefix"]).difference(set(geolocations_df6["geolocation_zip_code_prefix"])))
zip_cust_list.sort()
print(f"Number of missing zipcodes: {len(zip_cust_list)}")

lst2 = geolocations_df6["geolocation_zip_code_prefix"].tolist()

if not zip_cust_list:
    print("All customers' zipcodes are now present in the Geolocations dataframe.")
else:
    print(f"The following {len(zip_cust_list)} zipcodes are still missing from the Geolocations dataframe: {zip_cust_list}")
    
    # Iteratively repeat the procedure until no further missing zipcodes are found
    while zip_cust_list:
        geolocations_df5 = geolocations_df6
        
        subs_zip3 = []
        for missing_zip3 in zip_cust_list:
            subs_zip3.append(closest(lst2, missing_zip3))
        
        cust_dict2 = dict(zip(subs_zip3, zip_cust_list))
        
        cust_zip_df2 = geolocations_df5[geolocations_df5["geolocation_zip_code_prefix"].isin(subs_zip3)]
        cust_zip_df3 = cust_zip_df2.copy()
        cust_zip_df3["geolocation_zip_code_prefix"] = cust_zip_df3["geolocation_zip_code_prefix"].map(cust_dict2)
        
        geolocations_df6 = pd.concat([geolocations_df5, cust_zip_df3])
        geolocations_df6.reset_index(drop=True)
        print("Looping...")
        
        zip_cust_list = list(set(customers_df4["customer_zip_code_prefix"]).difference(set(geolocations_df6["geolocation_zip_code_prefix"])))
        
    print("All customers' zipcodes are now present in the Geolocations dataframe.")
   

In [ ]:
# The no. of unique geolocations_zip_code_prefix has increased by 156 (from Customers) to 19,173
geolocations_df6.nunique()

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
census_spark_df = spark.createDataFrame(census_df3)

# Write Spark DataFrame to delta
census_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/census")

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
customers_spark_df = spark.createDataFrame(customers_df4)

# Write Spark DataFrame to CSV
customers_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/customers")

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
sellers_spark_df = spark.createDataFrame(sellers_df4)

# Write Spark DataFrame to delta
sellers_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/sellers")

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
geolocations_spark_df = spark.createDataFrame(geolocations_df6)

# Write Spark DataFrame to delta
geolocations_spark_df.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/geolocations")

## Transform Reviews Data

In [ ]:
#Step 1 Convert relevent spark dataframes to pandas dataframe and EDA

reviews_df1 = pd.DataFrame(spark.read.format("parquet").load(input_path_reviews,header=True).collect())

# Check data quantity
print(reviews_df1.shape)
# Display first few rows
print(reviews_df1.head())
#Data types of each column and the number of non-null values.
print(reviews_df1.info())


In [ ]:
#Correct column names, data types and standardize format.

columns = {0: "review_id", 1: "order_id", 2:"review_score",3: "review_comment_title", 4: "review_comment_message", 5: "review_creation_date",6:"review_answer_timestamp"}
reviews_df1.rename(columns=columns, inplace=True)

# Custom function to parse dates with flexible format handling
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str)
    except ValueError:
        return pd.NaT

# Convert the data type of the 'review_id','order_id','review_comment_title','review_comment_message' columns to string
reviews_df1[['review_id','order_id','review_comment_title','review_comment_message']] = reviews_df1[['review_id','order_id','review_comment_title','review_comment_message']].astype('string')

# Convert the 'review_creation_date','review_answer_timestamp' columns to datetime format using custom parser
reviews_df1[['review_creation_date', 'review_answer_timestamp']] = reviews_df1[
    ['review_creation_date', 'review_answer_timestamp']
].applymap(parse_date)

reviews_df1.info()

In [ ]:
#Fill empty rows with Null and drop duplicate reviews

# Fill empty rows in 'review_comment_title' column with 'NULL'
reviews_df1['review_comment_title'] = reviews_df1['review_comment_title'].fillna('NULL')
reviews_df1['review_comment_message'] = reviews_df1['review_comment_message'].fillna('NULL')

# Replace 'NaN' with 'NULL' in the 'review_comment_title' column
reviews_df1['review_comment_title'] = reviews_df1['review_comment_title'].replace('nan', 'NULL')
reviews_df1['review_comment_message'] = reviews_df1['review_comment_message'].replace('nan', 'NULL')

# Keep only the rows where 'review_id' is unique
reviews_df1.drop_duplicates(subset=['review_id'], inplace=True)

%store reviews_df1
reviews_df1.info()

In [ ]:
# Further cleaning of review_comment_message column to form reviews_text

# Create a new DataFrame named reviews_text as a copy of reviews_full
reviews_text= reviews_df1.copy()

# Filter out rows where review_comment_message is blank
reviews_text = reviews_text[reviews_text['review_comment_message'].notnull()]

# Filter out rows where review_comment_message contains only non-alphabetic characters
reviews_text = reviews_text[reviews_text['review_comment_message'].str.replace(r'[^a-zA-Z]', '', regex=True).str.len() > 0]

# Define a function to count the number of words in a string
def count_words(text):
    if pd.isna(text):
        return 0
    words = text.split()
    return len(words)

# Apply the function to count the number of words in each review_comment_message
reviews_text['num_words'] = reviews_text['review_comment_message'].apply(count_words)

# Filter out rows where the number of words is less than or equal to 2
reviews_text = reviews_text[reviews_text['num_words'] > 2]

# Drop the 'num_words' column as it's no longer needed
reviews_text = reviews_text.drop(columns=['num_words'])

# Randomly sample 1000 rows from reviews_text
reviews_text = reviews_text.sample(n=1000, random_state=42)

# Display the shape of new DataFrame
print(reviews_text.shape)

In [ ]:
# Convert Pandas DataFrame to Spark DataFrame
order_reviews_spark_full = spark.createDataFrame(reviews_df1)
order_reviews_spark_text = spark.createDataFrame(reviews_text)

# Write Spark DataFrame to delta
order_reviews_spark_full.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/order_reviews_full")
order_reviews_spark_text.write.format("delta").mode("overwrite").option("header", "true").save("dbfs:/mnt/olist-silver/order_reviews_fortrans")